# Visualizing Results from Paraclassifier Training

## Libraries

In [ ]:
from generator import ParaclassifierGenerator
from keras.models import load_model
from keras import backend as K

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import os
import gc

%matplotlib inline

## Create Directory if not exists

In [ ]:
base_path = os.path.join('docs', 'paraclassifier')
if not os.path.exists(base_path):
    os.mkdir(base_path)

## Results from Training

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3)
fig.suptitle('Training Results', fontsize=20)
fig.set_figwidth(16)
fig.set_figheight(9)

ax[0, 0].set_title('MSE Loss/Epoch with ResNet 50')
df = pd.read_csv('logs/para_resnet.csv')
ax[0, 0].plot(df['loss'], label='Train')
ax[0, 0].plot(df['val_loss'], label='Validation')
del df
ax[0, 0].legend()

ax[0, 1].set_title('MSE Loss/Epoch with Inception V3')
df = pd.read_csv('logs/para_inception.csv')
ax[0, 1].plot(df['loss'], label='Train')
ax[0, 1].plot(df['val_loss'], label='Validation')
del df
ax[0, 1].legend()

ax[0, 2].set_title('MSE Loss/Epoch with InceptionResnet V2')
df = pd.read_csv('logs/para_inceptionresnet.csv')
ax[0, 2].plot(df['loss'], label='Train')
ax[0, 2].plot(df['val_loss'], label='Validation')
del df
ax[0, 2].legend()

ax[1, 0].set_title('MSE Loss/Epoch with Xception')
df = pd.read_csv('logs/para_xception.csv')
ax[1, 0].plot(df['loss'], label='Train')
ax[1, 0].plot(df['val_loss'], label='Validation')
del df
ax[1, 0].legend()

ax[1, 1].set_title('MSE Loss/Epoch with DenseNet 121')
df = pd.read_csv('logs/para_densenet.csv')
ax[1, 1].plot(df['loss'], label='Train')
ax[1, 1].plot(df['val_loss'], label='Validation')
del df
ax[1, 1].legend()

ax[1, 2].set_title('MSE Loss/Epoch with VGG 19')
df = pd.read_csv('logs/para_vgg.csv')
ax[1, 2].plot(df['loss'], label='Train')
ax[1, 2].plot(df['val_loss'], label='Validation')
del df
ax[1, 2].legend()

gc.collect()
plt.savefig(os.path.join(base_path, 'results.png'))

![Results](docs/paraclassifier/results.png)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
fig.suptitle('Comparitive Performance', fontsize=20)
fig.set_figwidth(21)
fig.set_figheight(9)

ax[0].set_title('Validation MSE Loss/Epoch', fontsize=15)
ax[1].set_title('Training MSE Loss/Epoch', fontsize=15)

df = pd.read_csv('logs/para_resnet.csv')
ax[0].plot(df['val_loss'], label='ResNet 50')
ax[1].plot(df['loss'], label='ResNet 50')
del df

df = pd.read_csv('logs/para_inceptionresnet.csv')
ax[0].plot(df['val_loss'], label='InceptionResNet V2')
ax[1].plot(df['loss'], label='InceptionResNet V2')
del df

df = pd.read_csv('logs/para_inception.csv')
ax[0].plot(df['val_loss'], label='Inception V3')
ax[1].plot(df['loss'], label='Inception V3')
del df

df = pd.read_csv('logs/para_xception.csv')
ax[0].plot(df['val_loss'], label='Xception')
ax[1].plot(df['loss'], label='Xception')
del df

df = pd.read_csv('logs/para_densenet.csv')
ax[0].plot(df['val_loss'], label='DenseNet 121')
ax[1].plot(df['loss'], label='DenseNet 121')
del df

df = pd.read_csv('logs/para_vgg.csv')
ax[0].plot(df['val_loss'], label='VGG 19')
ax[1].plot(df['loss'], label='VGG 19')
del df

ax[0].legend()
ax[1].legend()

gc.collect()
plt.savefig(os.path.join(base_path, 'performance.png'))

![Performance](docs/paraclassifier/performance.png)

## Test Data Set

In [ ]:
results = []
test = pd.read_csv(os.path.join('meta', 'para_test.csv')).values.tolist()

### Function to fetch Model

In [ ]:
def build_paraclassifier(key):
    model = None
    model_d = {'densenet': 'dn121', 'inceptionresnet': 'irv2',
               'inception': 'iv3', 'resnet': 'r50',
               'vgg': 'vgg', 'xception': 'x'}
    if key == 'densenet':
        from models.densenet import get_paraclassifier
    elif key == 'inceptionresnet':
        from models.inceptionresnet import get_paraclassifier
    elif key == 'inception':
        from models.inception import get_paraclassifier
    elif key == 'resnet':
        from models.resnet import get_paraclassifier
    elif key == 'vgg':
        from models.vgg import get_paraclassifier
    else:
        from models.xception import get_paraclassifier

    model_path = os.path.join('weights', 'para_{}.h5'.format(key))
    model = load_model(model_path)
    return model

### ResNet 50

In [ ]:
result = []
df = pd.read_csv(os.path.join('logs', 'para_resnet.csv'))
result.append('resnet')
result.append(df.loc[df['val_loss'].idxmin()]['categorical_accuracy'])
result.append(df.loc[df['val_loss'].idxmin()]['val_categorical_accuracy'])
del df

K.clear_session()
model = build_paraclassifier('resnet')
test_generator = ParaclassifierGenerator('resnet', test, 32, True)
pred = model.evaluate_generator(test_generator)
result.append(pred[1])
results.append(result)

del model
del test_generator
_ = gc.collect()

### Inception

In [ ]:
result = []
df = pd.read_csv(os.path.join('logs', 'para_inception.csv'))
result.append('inception')
result.append(df.loc[df['val_loss'].idxmin()]['categorical_accuracy'])
result.append(df.loc[df['val_loss'].idxmin()]['val_categorical_accuracy'])
del df

K.clear_session()
model = build_paraclassifier('inception')
test_generator = ParaclassifierGenerator('inception', test, 32, True)
pred = model.evaluate_generator(test_generator)
result.append(pred[1])
results.append(result)

del model
del test_generator
_ = gc.collect()

### InceptionResnet

In [ ]:
result = []
df = pd.read_csv(os.path.join('logs', 'para_inceptionresnet.csv'))
result.append('inceptionresnet')
result.append(df.loc[df['val_loss'].idxmin()]['categorical_accuracy'])
result.append(df.loc[df['val_loss'].idxmin()]['val_categorical_accuracy'])
del df

K.clear_session()
model = build_paraclassifier('inceptionresnet')
test_generator = ParaclassifierGenerator('inceptionresnet', test, 32, True)
pred = model.evaluate_generator(test_generator)
result.append(pred[1])
results.append(result)

del model
del test_generator
_ = gc.collect()

### DenseNet

In [ ]:
result = []
df = pd.read_csv(os.path.join('logs', 'para_densenet.csv'))
result.append('densenet')
result.append(df.loc[df['val_loss'].idxmin()]['categorical_accuracy'])
result.append(df.loc[df['val_loss'].idxmin()]['val_categorical_accuracy'])
del df

K.clear_session()
model = build_paraclassifier('densenet')
test_generator = ParaclassifierGenerator('densenet', test, 32, True)
pred = model.evaluate_generator(test_generator)
result.append(pred[1])
results.append(result)

del model
del test_generator
_ = gc.collect()

### Xception

In [ ]:
result = []
df = pd.read_csv(os.path.join('logs', 'para_xception.csv'))
result.append('xception')
result.append(df.loc[df['val_loss'].idxmin()]['categorical_accuracy'])
result.append(df.loc[df['val_loss'].idxmin()]['val_categorical_accuracy'])
del df

K.clear_session()
model = build_paraclassifier('xception')
test_generator = ParaclassifierGenerator('xception', test, 32, True)
pred = model.evaluate_generator(test_generator)
result.append(pred[1])
results.append(result)

del model
del test_generator
_ = gc.collect()

### VGG

In [ ]:
result = []
df = pd.read_csv(os.path.join('logs', 'para_vgg.csv'))
result.append('vgg')
result.append(df.loc[df['val_loss'].idxmin()]['categorical_accuracy'])
result.append(df.loc[df['val_loss'].idxmin()]['val_categorical_accuracy'])
del df

K.clear_session()
model = build_paraclassifier('vgg')
test_generator = ParaclassifierGenerator('vgg', test, 32, True)
pred = model.evaluate_generator(test_generator)
result.append(pred[1])
results.append(result)

del model
del test_generator
_ = gc.collect()

In [ ]:
results

In [ ]:
df = pd.DataFrame.from_records(results, columns=['Name', 'Training Accuracy',
                                                                     'Validation Accuracy',
                                                                     'Test Accuracy'])
print(df)
df.to_csv(os.path.join('meta', 'para_results.csv'))